### TIME SERIES

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [32]:
df = pd.read_csv('exchange_rate(3).csv')


FileNotFoundError: [Errno 2] No such file or directory: 'exchange_rate(3).csv'

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
z=df.corr(numeric_only=True)        # Correlation matrix
z

In [ ]:
### missing values
df.isnull().sum()

In [ ]:
### Plot the time series
plt.figure(figsize=(10,6))
plt.plot(df['Ex_rate'], label=' Exchange Rate')
plt.title('Exchange Rate Over Time')
plt.xlabel('Date')
plt.ylabel('Exchange Rate')
plt.legend()
plt.show()

In [ ]:
from scipy.stats import zscore
df['z_score'] = zscore(df['Ex_rate'])
df = df[df['z_score'].abs() < 3]
df.drop(columns=['z_score'], inplace=True)

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
plot_acf(df['Ex_rate'], lags=50)
plot_pacf(df['Ex_rate'], lags=50)
plt.show()

In [ ]:
from statsmodels.tsa.stattools import adfuller
result = adfuller(df['Ex_rate'])
print(f'ADF Statistic: {result[0]}')
print(f'p-value: {result[1]}')

In [ ]:
p = 1
d = 1
q = 1

In [ ]:
from statsmodels.tsa.arima.model import ARIMA            # developing arima model
model = ARIMA(df['Ex_rate'], order=(p, d, q))
arima_result = model.fit()
print(arima_result.summary())

In [ ]:
arima_result.plot_diagnostics(figsize=(12, 8))                  # arima model visualization
plt.show()

In [ ]:
print(df.index)
print(type(df.index))

In [ ]:
### arima forecast visualization
forecast = arima_result.get_forecast(steps=30)
forecast_index = pd.date_range(df.index[-1], periods=30, freq='D')

forecast_mean = forecast.predicted_mean
forecast_ci = forecast.conf_int()

plt.figure(figsize=(10, 6))
plt.plot(df['Ex_rate'], label='Observed')
plt.plot(forecast_index, forecast_mean, label='Forecast')
plt.fill_between(forecast_index, forecast_ci.iloc[:, 0], forecast_ci.iloc[:, 1], color='gray', alpha=0.2)
plt.title('ARIMA Forecast')
plt.legend()
plt.show()     

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
model_hw = ExponentialSmoothing(df['Ex_rate'], trend='add', seasonal=None)
hw_result = model_hw.fit()
print(hw_result.summary())

In [ ]:
forecast_hw = hw_result.forecast(steps=30)
### exponential smoothing forecast visualization  
plt.figure(figsize=(10, 6))
plt.plot(df['Ex_rate'], label='Observed')
plt.plot(forecast_index, forecast_hw, label='Holt-Winters Forecast', color='orange')
plt.title('Exponential Smoothing Forecast')
plt.legend()
plt.show()  

In [ ]:
actual = df['Ex_rate'][-30:]

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
mae_arima = mean_absolute_error(actual, forecast_mean)
# Calculate RMSE manually
rmse_arima = np.sqrt(mean_squared_error(actual, forecast_mean))
mae_hw = mean_absolute_error(actual, forecast_hw)
# Calculate RMSE manually
rmse_hw = np.sqrt(mean_squared_error(actual, forecast_hw))

In [ ]:
print(f'ARIMA - MAE: {mae_arima}, RMSE: {rmse_arima}')
print(f'Holt-Winters - MAE: {mae_hw}, RMSE: {rmse_hw}')